In [ ]:
import torch
import random
from PIL import Image
import os
from torchvision.datasets import CIFAR10, CIFAR100, CocoCaptions, ImageNet
import slip_models
from tokenizer import SimpleTokenizer

import torchvision.transforms as T
from torchvision import datasets, transforms


from tqdm import tqdm
import json
import matplotlib.pyplot as plt

import numpy as np
%pip install ipywidgets
%pip install update tqdm%matplotlib inline


In [ ]:
# !/export/home/google-cloud-sdk/bin/gsutil -m cp gs://sfr-tpu-us-east1-research/bwallace/ckpts/models/multimodel_clip/parallel_vision_standard_text/viz_parallel_text_standard_head_dim_64_per_expert_norm_bs_1024_epoch_30.ckpt ckpts/viz_parallel_text_standard_head_dim_64_per_expert_norm_bs_1024_epoch_30.ckpt
!/export/home/google-cloud-sdk/bin/gsutil -m cp gs://sfr-tpu-us-east1-research/bwallace/ckpts/models/multimodel_clip/parallel_vision_standard_text/viz_parallel_text_standard_head_dim_64_top1_expert_loss_no_entroy_bs_1024_epoch_6.ckpt ckpts/viz_parallel_text_standard_head_dim_64_top1_expert_loss_no_entroy_bs_1024_epoch_6.ckpt


In [ ]:
preprocess = T.Compose(
                    [
                        T.Resize(256),
                        T.CenterCrop(224),
                        T.ToTensor(),
                        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                    ]
                        )

grayscale_preprocess = T.Compose(
                    [
                        T.Resize(256),
                        T.CenterCrop(224),
                        T.ToTensor(),
                        T.Normalize(mean=[np.average([0.485, 0.456, 0.406])], std=[np.average([0.229, 0.224, 0.225])]),
                        T.Lambda(lambda x: x.repeat(3, 1, 1) ),
                    ]
                        )

In [ ]:
grayscale_preprocess

In [ ]:
# model = slip_models.CLIP_VITB16()
# ckpt = torch.load('ckpts/clip_vit_epoch_35.ckpt', map_location='cpu')

model = slip_models.VisionParallelTextStandard(64, 64)
ckpt = torch.load('ckpts/viz_parallel_text_standard_head_dim_64_top1_expert_loss_no_entroy_bs_1024_epoch_6.ckpt', map_location='cpu')
per_group_norm = True
# ckpt = torch.load('tmpp_save/test_epoch_4.ckpt', map_location='cpu')
# model = slip_models.VisionParallelTextStandard(64, 8)
# ckpt = torch.load('ckpts/viz_parallel_text_standard_bs_1024_epoch_35.ckpt', map_location='cpu')
# per_group_norm = False

# model = slip_models.CLIP_VITB16(num_prompt_tokens=64, num_text_outputs=1000)
# ckpt = torch.load('ckpts/epoch_30_prompted_clip_may_27.ckpt', map_location='cpu')


# model.load_state_dict({k.replace('module.',''):v for k,v in ckpt["model"].items()})
model.load_state_dict(ckpt["model"])

model.eval()
model = model.cuda()

In [ ]:
# cc_model = slip_models.CLIP_VITB16() # ssl_mlp_dim=4096, ssl_emb_dim=256)
# cc_model =  slip_models.CLIP_VITB16()
# ckpt = torch.load('ckpts/clip_vit_epoch_35.ckpt', map_location='cpu')
# ckpt = torch.load('ckpts/retest_baseline_jul_27_epoch_5.ckpt', map_location='cpu')



cc_model =  slip_models.CLIP_ResNet18()
# ckpt = torch.load('ckpts/clip_vit_epoch_35.ckpt', map_location='cpu')
ckpt = torch.load('ckpts/resnet18_baseline_jul_27_epoch_35.ckpt', map_location='cpu')

# ckpt = torch.load('ckpts/slip_base_100ep.pt', map_location='cpu')
# cc_model.load_state_dict({k.replace('module.',''):v for k,v in ckpt["state_dict"].items()})
cc_model.load_state_dict({k.replace('module.',''):v for k,v in ckpt["model"].items()})
cc_model = cc_model.cuda()

In [ ]:
tokenizer = SimpleTokenizer()

In [ ]:
# image = preprocess(Image.open("pics/golden-retriever.png")).unsqueeze(0).to(device)
# image = preprocess(Image.open("pics/CLIP.png")).unsqueeze(0).to(device)
base_text = ["a diagram", "a dog", "a cat"]
model.eval()
dog_image = preprocess(Image.open("pics/golden-retriever.png"))
diagram_image = preprocess(Image.open("pics/CLIP.png").convert("RGB"))
cat_image = preprocess(Image.open("pics/cat.jpg"))
images = torch.stack([diagram_image, dog_image, cat_image]).cuda()

text = tokenizer([f"a picture of {s}" for s in base_text]).cuda()

with torch.no_grad():
    image_features = model.encode_image(images)
    text_features = model.encode_text(text)
    
    if per_group_norm:
        image_features = image_features.view(image_features.shape[0], 64, -1)
        text_features = text_features.view(image_features.shape[0], 64, -1)
    image_features = image_features / image_features.norm(dim=-1, keepdim=True)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)
    
    image_features = image_features.flatten(start_dim=1)
    text_features = text_features.flatten(start_dim=1)
    
    logits_per_image = model.logit_scale.exp() * image_features @ text_features.t()
    logits_per_text = logits_per_image.t()
    # logits_per_image, logits_per_text = model(images, text)
    per_image_probs = logits_per_image.softmax(dim=-1).cpu().numpy()
    per_text_probs = logits_per_text.softmax(dim=-1).cpu().numpy()

print("(Per Image) Label probs:\n", per_image_probs)  # prints: [[0.9927937  0.00421068 0.00299572]]
print("(Per Text) Label probs:\n", per_text_probs)  # prints: [[0.9927937  0.00421068 0.00299572]]

In [ ]:
image_features @ text_features.t()

In [ ]:
image_features @ image_features.t()

In [ ]:
grouped_if = image_features.view(3, 64, 64)
grouped_sims = torch.bmm(grouped_if, grouped_if.transpose(1, 2))
print(grouped_sims.shape)

In [ ]:
grouped_sims[0,0]

In [ ]:
# image = preprocess(Image.open("pics/golden-retriever.png")).unsqueeze(0).to(device)
# image = preprocess(Image.open("pics/CLIP.png")).unsqueeze(0).to(device)
base_text = ["a diagram", "a dog", "a cat"]
cc_model.eval()
dog_image = preprocess(Image.open("pics/golden-retriever.png"))
diagram_image = preprocess(Image.open("pics/CLIP.png").convert("RGB"))
cat_image = preprocess(Image.open("pics/cat.jpg"))
images = torch.stack([diagram_image, dog_image, cat_image]).cuda()

text = tokenizer([f"a picture of {s}" for s in base_text]).cuda()

with torch.no_grad():
    image_features = cc_model.encode_image(images)
    text_features = cc_model.encode_text(text)
    
    image_features = image_features / image_features.norm(dim=-1, keepdim=True)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)
    
    
    logits_per_image = cc_model.logit_scale.exp() * image_features @ text_features.t()
    # logits_per_image, logits_per_text = model(images, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

print("Label probs:", probs)  # prints: [[0.9927937  0.00421068 0.00299572]]

In [ ]:
image_features @ text_features.t()

# Basic ImageNet and CIFAR checks

In [ ]:
per_group_norm=False

# Download the dataset
cifar10 = CIFAR10(root="/tmp/", transform=preprocess, download=True, train=False)
loader = torch.utils.data.DataLoader(cifar10, batch_size=32, shuffle=True)
# text_inputs = torch.stack([tokenizer(f"a photo of a {c}") for c in cifar10.classes]).cuda()


text_inputs = torch.stack([tokenizer(f"a photo of a {c}") for c in cifar10.classes]).cuda()


num_correct = 0
num_seen = 0
with torch.no_grad():
    #  text_features = model.encode_text(text_inputs)
    for imgs, targets in loader:
        imgs = imgs.cuda()
        targets = targets.cuda()
        
        # image_features = model.encode_image(imgs)
        # results = model(imgs, text_inputs, lang_prompt_viz=True, sharded_computation=False)
        # results = cc_model(imgs, text_inputs)
        image_features = cc_model.encode_image(imgs)
        text_features = cc_model.encode_text(text_inputs)

        if per_group_norm:
            image_features = image_features.view(image_features.shape[0], 64, -1)
            text_features = text_features.view(text_features.shape[0], 64, -1)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)

        image_features = image_features.flatten(start_dim=1)
        text_features = text_features.flatten(start_dim=1)
        # image_features = (image_features - image_features.mean(0)) / torch.sqrt(image_features.var(0))
        # print("Using barlow"); text_features = (text_features - text_features.mean(0)) / torch.sqrt(text_features.var(0)) 
        # image_features = (image_features - coco_image_mean) / torch.sqrt(coco_image_var)
        # print("Using barlow"); text_features = (text_features - coco_text_mean) / torch.sqrt(coco_text_var)  
        
    
        similarity = (image_features @ text_features.T).softmax(dim=-1)
        max_sims = similarity.max(dim=-1)[0]
        # print(max_sims.min(), max_sims.max(), max_sims.mean())
        num_correct += (similarity.argmax(dim=-1)==targets).sum().item()  
        num_seen += imgs.shape[0]
        curr_acc = num_correct / num_seen
        print(f"{num_seen} / {len(cifar10)} // Current Acc: {curr_acc}")
        if num_seen > 100000: break
acc = num_correct / num_seen
print(f"(PARTIAL) Final Acc {acc}")

In [ ]:
# Download the dataset
cifar10 = CIFAR10(root="/tmp/", transform=preprocess, download=True, train=False)
loader = torch.utils.data.DataLoader(cifar10, batch_size=32, shuffle=True)
# text_inputs = torch.stack([tokenizer(f"a photo of a {c}") for c in cifar10.classes]).cuda()


text_inputs = torch.stack([tokenizer(f"a photo of a {c}") for c in cifar10.classes]).cuda()


num_correct = 0
num_seen = 0
with torch.no_grad():
    #  text_features = model.encode_text(text_inputs)
    for imgs, targets in loader:
        imgs = imgs.cuda()
        targets = targets.cuda()
        
        # image_features = model.encode_image(imgs)
        # results = model(imgs, text_inputs, lang_prompt_viz=True, sharded_computation=False)
        # results = cc_model(imgs, text_inputs)
        image_features = model.encode_image(imgs)
        text_features = model.encode_text(text_inputs)

        if per_group_norm:
            image_features = image_features.view(image_features.shape[0], 64, -1)
            text_features = text_features.view(text_features.shape[0], 64, -1)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)

        image_features = image_features.flatten(start_dim=1)
        text_features = text_features.flatten(start_dim=1)
        # image_features = (image_features - image_features.mean(0)) / torch.sqrt(image_features.var(0))
        # print("Using barlow"); text_features = (text_features - text_features.mean(0)) / torch.sqrt(text_features.var(0)) 
        # image_features = (image_features - coco_image_mean) / torch.sqrt(coco_image_var)
        # print("Using barlow"); text_features = (text_features - coco_text_mean) / torch.sqrt(coco_text_var)  
        
    
        similarity = (image_features @ text_features.T).softmax(dim=-1)
        max_sims = similarity.max(dim=-1)[0]
        # print(max_sims.min(), max_sims.max(), max_sims.mean())
        num_correct += (similarity.argmax(dim=-1)==targets).sum().item()  
        num_seen += imgs.shape[0]
        curr_acc = num_correct / num_seen
        print(f"{num_seen} / {len(cifar10)} // Current Acc: {curr_acc}")
        if num_seen > 100000: break
acc = num_correct / num_seen
print(f"(PARTIAL) Final Acc {acc}")

In [ ]:
imagenet = ImageNet(root="/export/share/datasets/vision/imagenet", transform=preprocess, split='val')
loader = torch.utils.data.DataLoader(imagenet, batch_size=32, num_workers=4, shuffle=True)
text_inputs = torch.stack([tokenizer(f"a photo of a {c}") for c in imagenet.classes]).cuda()

num_correct = 0
num_seen = 0
with torch.no_grad():
    text_features = cc_model.encode_text(text_inputs)
    if per_group_norm:
        text_features = text_features.view(text_features.shape[0], 64, -1)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)
    text_features = text_features.flatten(start_dim=1)
    # print("Using barlow"); text_features = (text_features - text_features.mean(0)) / torch.sqrt(text_features.var(0))
    # print("Using barlow"); text_features = (text_features - coco_text_mean) / torch.sqrt(coco_text_var)  
    
    for imgs, targets in loader:
        imgs = imgs.cuda()
        targets = targets.cuda()
        
        image_features = cc_model.encode_image(imgs)
        if per_group_norm:
            image_features = image_features.view(image_features.shape[0], 64, -1)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        image_features = image_features.flatten(start_dim=1)
        # print("using barlow"); image_features = (image_features - image_features.mean(0)) / torch.sqrt(image_features.var(0))
        # results = model(imgs, text_inputs, lang_prompt_viz=True, sharded_computation=False)
        # image_features = results['image_embed']
        # text_features = results['text_embed']
        # image_features = (image_features - coco_image_mean) / torch.sqrt(coco_image_var) 
        
        
        similarity = (image_features @ text_features.T).softmax(dim=-1)
        num_correct += (similarity.argmax(dim=-1)==targets).sum().item()  
        max_sims = similarity.max(dim=-1)[0]
        # print(max_sims.min(), max_sims.max(), max_sims.mean())
        num_seen += imgs.shape[0]
        curr_acc = num_correct / num_seen
        print(f"{num_seen} / {len(imagenet)} // Current Acc: {curr_acc}")
        if num_seen > 100000: break
        
print(f"(PARTIAL) Final Acc {curr_acc}")

In [ ]:
similarity.shape

In [ ]:
imagenet = ImageNet(root="/export/share/datasets/vision/imagenet", transform=preprocess, split='val')
loader = torch.utils.data.DataLoader(imagenet, batch_size=32, num_workers=4, shuffle=True)
text_inputs = torch.stack([tokenizer(f"a photo of a {c}") for c in imagenet.classes]).cuda()

num_correct = 0
num_seen = 0
with torch.no_grad():
    text_features = model.encode_text(text_inputs)
    if per_group_norm:
        text_features = text_features.view(text_features.shape[0], 64, -1)
    text_features = text_features / text_features.norm(dim=-1, keepdim=True)
    text_features = text_features.flatten(start_dim=1)
    # print("Using barlow"); text_features = (text_features - text_features.mean(0)) / torch.sqrt(text_features.var(0))
    # print("Using barlow"); text_features = (text_features - coco_text_mean) / torch.sqrt(coco_text_var)  
    
    for imgs, targets in loader:
        imgs = imgs.cuda()
        targets = targets.cuda()
        
        image_features = model.encode_image(imgs)
        if per_group_norm:
            image_features = image_features.view(image_features.shape[0], 64, -1)
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        image_features = image_features.flatten(start_dim=1)
        # print("using barlow"); image_features = (image_features - image_features.mean(0)) / torch.sqrt(image_features.var(0))
        # results = model(imgs, text_inputs, lang_prompt_viz=True, sharded_computation=False)
        # image_features = results['image_embed']
        # text_features = results['text_embed']
        # image_features = (image_features - coco_image_mean) / torch.sqrt(coco_image_var) 
        
        
        similarity = (image_features @ text_features.T).softmax(dim=-1)
        num_correct += (similarity.argmax(dim=-1)==targets).sum().item()  
        max_sims = similarity.max(dim=-1)[0]
        # print(max_sims.min(), max_sims.max(), max_sims.mean())
        num_seen += imgs.shape[0]
        curr_acc = num_correct / num_seen
        print(f"{num_seen} / {len(imagenet)} // Current Acc: {curr_acc}")
        if num_seen > 1000: break
        
print(f"(PARTIAL) Final Acc {curr_acc}")

In [ ]:


def dataset_constructor(name, split, transform):
    """
    inputs
        name: string that we'll run if/else on
        split: "train" or "val" for now
    """
    assert split in ['train', 'val']

    # Need to figure out way to override the getitem call for strong weak augmentatin
    if name == 'cifar10':
        # Has class names
        return datasets.CIFAR10(root=os.path.expanduser("~/.cache"),
                                download=True,
                                train=(split=='train'),
                               transform=transform)
    elif name == 'cifar100':
        # Has class names
        return datasets.CIFAR100(root=os.path.expanduser("~/.cache"),
                                download=True,
                                train=(split=='train'),
                                transform=transform)

    elif name == 'svhn':
        # Doesn't have class names
        return datasets.SVHN(root=os.path.expanduser("~/.cache"),
                             download=True,
                             transform=transform,
                             split=split)
    elif name == 'gtsrb': #####
        assert split=='val'
        return datasets.ImageFolder(root="/export/share/bwallace/datasets/gtsrb/pytorch_format",
                                   transform=transform)
        return datasets.GTSRB(root=os.path.expanduser("~/.cache"), split='test', download=True)
    elif name == 'food101':
        # name is in .classes
        assert split=='val'
        return datasets.ImageFolder(root="/export/share/bwallace/datasets/food101/images/",
                   transform=transform)
    elif name == 'merced':
        # name is in .classes
        assert split=='val'
        return datasets.ImageFolder(root="/export/share/bwallace/datasets/UCMerced_LandUse/Images/",
                   transform=transform)
    elif name == 'mnist':
        assert split=='val'
        return datasets.MNIST(root=os.path.expanduser("~/.cache"), train=False, download=True, transform=transform)
    elif name == 'flowers': ####
        assert split=='val'
        print("NOTE PROPER METRIC IS MEAN PER CLASS")
        return datasets.ImageFolder(root="/export/share/bwallace/datasets/flowers/test/",
                                    transform=transform)
    elif name == 'aircraft': ####
        assert split=='val'
        print("NOTE PROPER METRIC IS MEAN PER CLASS")
        return datasets.ImageFolder(root="/export/share/bwallace/datasets/aircraft/data/test_pytorch_format/",
                                   transform=transform)
    elif name == 'cars':
        assert split=='val'
        return datasets.ImageFolder(root="/export/share/datasets/vision/stanford_cars/car_data/test/",
                                   transform=transform)
    elif name == 'eurosat':
        assert split=='val'
        return datasets.ImageFolder(root="/export/share/datasets/vision/euro_sat/2750/",
                                   transform=transform)
    elif name == 'dtd':
        # name is in .classes
        return datasets.ImageFolder(root=f"/export/share/bwallace/datasets/dtd/{split}",
                   transform=transform)
    elif name == 'cub':
        # name is in .classes
        return datasets.ImageFolder(root=f"/export/share/bwallace/datasets/CUB_2011_formatted/{split}",
                   transform=transform)
    elif name == 'places365':
        # class names works in 
        return datasets.Places365(
                    root="/export/share/datasets/vision/Places365/",
                    split='train-standard' if split=='train' else split,
                   transform=transform)

    elif name == 'imagenet':
        # Doesn't have class names, have separate call
        return datasets.ImageNet(
                    root="/export/share/datasets/vision/imagenet/",
                    split=split,
                   transform=transform)
    elif name == 'imagenet_val':
        assert split=='train'
        # Doesn't have class names, only use is having Imagenet val be trainset 
        return datasets.ImageNet(
                    root="/export/share/datasets/vision/imagenet/",
                    split='val',
                   transform=transform)
    else:
        raise NotImplementedError


def get_imagenet_class_dict():
        idx_to_word_id_and_name_tuple = json.load(open('imagenet_class_index.json'))
        word_id_to_name_and_idx = {v[0]:(v[1],int(k))
                           for k,v in idx_to_word_id_and_name_tuple.items()}
        # e.g. n0023923939 to ('unicorn', 123)
        return word_id_to_name_and_idx

In [ ]:
def test_on_dataset(model, dataset_name, transform,
                   prompt_template=None,
                   test_components_mode='standard',
                   normalization='mask',
                   component_grouping=1):
    """
    Options for test components mode
        standard : standard
        test_all : scale through
        random_all : As all but random sorting
        
    Normalization is whether to normalize from all components or just ones in computation
    
    """
    dataset = dataset_constructor(dataset_name, 'val', transform)
    loader = torch.utils.data.DataLoader(dataset, batch_size=32, num_workers=4)
    
    with torch.no_grad():
        
        
        if prompt_template is not None:
            class_names = [prompt_template.format(c.replace("'","")) for c in dataset.classes]
            # print(class_names)
            text_inputs = torch.stack([tokenizer(txt) for txt in class_names]).cuda()
            text_features = model.encode_text(text_inputs)
            if normalization in ['mask', 'global']:
                text_features = text_features / text_features.norm(dim=-1, keepdim=True)
            elif normalization == 'grouped':
                text_features = text_features.view(text_features.shape[0],
                                                                -1,
                                                                component_grouping)
                text_features =  text_features / text_features.norm(dim=-1, keepdim=True)
                text_features = text_features.flatten(start_dim=1)
            else:
                raise NotImplementedError
        else:
            with open('templates.json') as f:
                all_templates = json.load(f)
            templates = all_templates[dataset_name]
            text_features = []
            class_names = [c.replace("'", "") for c in dataset.classes]
            for class_name in class_names:
                text_inputs = torch.stack([tokenizer(t.format(class_name)) for t in templates]).cuda()
                class_features = model.encode_text(text_inputs)
                # class_features = class_features / class_features.norm(dim=-1, keepdim=True)
                if normalization in ['mask', 'global']:
                    class_features = class_features / class_features.norm(dim=-1, keepdim=True)
                elif normalization == 'grouped':
                    class_features = class_features.view(class_features.shape[0],
                                                                    -1,
                                                                    component_grouping)
                    class_features =  class_features / class_features.norm(dim=-1, keepdim=True)
                    class_features = class_features.flatten(start_dim=1)
                else:
                    raise NotImplementedError
                # print(class_features.shape)
                class_features = class_features.mean(dim=0)
                # print(class_features.shape)
                if normalization in ['mask', 'global']:
                    class_features = class_features / class_features.norm(dim=-1, keepdim=False)
                    # text_features = text_features / text_features.norm(dim=-1, keepdim=True)
                elif normalization == 'grouped':
                    class_features = class_features.view(-1, component_grouping)
                    class_features =  class_features / class_features.norm(dim=-1, keepdim=True)
                    class_features = class_features.flatten()
                else:
                    raise NotImplementedError
                # print(class_features.shape)
                text_features.append(class_features)
            text_features = torch.stack(text_features)
            # print(text_features.shape)
                
        if test_components_mode in ['standard']:
            sorted_feature_variances, sorted_variance_idx = None, None
        elif test_components_mode in ['test_all', 'ensemble']:
            # grouping will return indices in blocks (idx //component_grouping will be repeated numbers in blocks of component_grouping)
            sorted_feature_variances, sorted_variance_idx = sorted_feature_component_variances(text_features,
                                                                                              group_size=component_grouping)
        elif test_components_mode in ['all_indiv']:
            sorted_variance_idx = torch.arange(text_features.shape[1])
        elif test_components_mode in ['random_all']:
            # Want random to be grouped as well
            sorted_feature_variances, sorted_variance_idx = sorted_feature_component_variances(text_features,
                                                                                              group_size=component_grouping)
            grouped_idx = torch.randperm(sorted_variance_idx.shape[0] // component_grouping)
            sorted_variance_idx = torch.cat([torch.arange(component_grouping).cuda() + (component_grouping*i) for i in grouped_idx])
            # print(sorted_variance_idx)
        else:
            raise NotImplementedError
        
        num_correct = 0 if test_components_mode =='standard' else torch.zeros(text_features.shape[1] // component_grouping)
        num_seen = 0
        for imgs, targets in tqdm(loader):
            imgs = imgs.cuda()
            targets = targets.cuda()

            image_features = model.encode_image(imgs)
            image_features = image_features / image_features.norm(dim=1, keepdim=True)
            # results = model(imgs, text_inputs, lang_prompt_viz=True, sharded_computation=False)
            # image_features = results['image_embed']
            # text_features = results['text_embed']


            if test_components_mode=="standard":
                if normalization == 'grouped':
                    image_features = image_features.view(image_features.shape[0],
                                                                    -1,
                                                                    component_grouping)
                    image_features =  image_features / image_features.norm(dim=-1, keepdim=True)
                    image_features = image_features.flatten(start_dim=1)
                similarity = (image_features @ text_features.T).softmax(dim=-1)
                num_correct += (similarity.argmax(dim=-1)==targets).sum().item() 
            elif test_components_mode in ["test_all", "random_all", "ensemble", "all_indiv"]:
                for num_components in range(component_grouping, text_features.shape[1]+1,
                                           component_grouping):
                    
                    start_idx = num_components - component_grouping if test_components_mode == 'all_indiv' else 0
                    idx_to_use = sorted_variance_idx[start_idx:num_components]
                    masked_text_features = text_features[:, idx_to_use]
                    masked_image_features = image_features[:, idx_to_use]
                    if normalization == 'mask':
                        masked_text_features =  masked_text_features / masked_text_features.norm(dim=-1, keepdim=True)
                        masked_image_features =  masked_image_features / masked_image_features.norm(dim=-1, keepdim=True)
                    elif normalization == 'global':
                        masked_image_features = masked_image_features / image_features.norm(dim=-1, keepdim=True)
                    elif normalization == 'grouped':
                        masked_text_features = masked_text_features.view(masked_text_features.shape[0],
                                                                        -1,
                                                                        component_grouping)
                        masked_text_features =  masked_text_features / masked_text_features.norm(dim=-1, keepdim=True)
                        masked_text_features = masked_text_features.flatten(start_dim=1)
                        masked_image_features = masked_image_features.view(masked_image_features.shape[0],
                                                                        -1,
                                                                        component_grouping)
                        masked_image_features =  masked_image_features / masked_image_features.norm(dim=-1, keepdim=True)
                        masked_image_features = masked_image_features.flatten(start_dim=1)
                    else:
                        raise NotImplementedError
                    if test_components_mode in ["test_all", "random_all", "all_indiv"]:
                        masked_similarity = (masked_image_features @ masked_text_features.T).softmax(dim=-1)
                        num_correct[(num_components//component_grouping)-1] += (masked_similarity.argmax(dim=-1)==targets).sum().item()
                    elif test_components_mode == 'ensemble':
                        # right now features are bs x (head_dim * num_components)
                        # put as bs x num_components x head_dim
                        # permute to num_components x bs x head_dim
                        # BMM against num_components x head_dim x bs
                        # get num_components bs x bs selection matrices
                        # print(masked_image_features.shape)
                        # print(masked_text_features.shape)
                        num_texts = masked_text_features.shape[0]
                        num_images = masked_image_features.shape[0]
                        masked_image_features = masked_image_features.view(num_images, -1, component_grouping).permute(1, 0, 2)
                        masked_text_features = masked_text_features.view(num_texts, -1, component_grouping).permute(1, 2, 0)
                        per_component_sims = torch.bmm(masked_image_features, masked_text_features) # num_components x num_images x num_texts
                        per_image_per_component_preds = per_component_sims.argmax(dim=-1) # num_components x num_images
                        # print(per_image_per_component_preds)
                        # print(per_image_per_component_preds.shape)
                        ensemble_pred = per_image_per_component_preds.cpu().mode(0)[0]
                        # print(ensemble_pred.shape, targets.shape)
                        num_correct[(num_components//component_grouping)-1] += (ensemble_pred == targets.cpu()).sum().item()
                        # BMM against bs x head_dim x num_components and get 
                        # bs x num_components x num_components
            else:
                 raise NotImplementedError
            # max_sims = similarity.max(dim=-1)[0]
            # print(max_sims.min(), max_sims.max(), max_sims.mean())
            num_seen += imgs.shape[0]
            curr_acc = num_correct / num_seen
    return curr_acc


def get_dataset_text_features(model, dataset_name, norm=True, print_class_names=False):
    dataset = dataset_constructor(dataset_name, 'val', None)
    class_names = [f"a photo of a {c}" for c in dataset.classes]
    if print_class_names:
            print(dataset.classes)
    text_inputs = torch.stack([tokenizer(txt) for txt in class_names]).cuda()
    with torch.no_grad():
        text_features = model.encode_text(text_inputs)
        if norm: text_features = text_features / text_features.norm(dim=1, keepdim=True)
    return text_features

def sorted_feature_component_variances(tensor, group_size=1):
    # NOTE: sorting max first so [0] is intuitively important
    feature_variances = tensor.var(dim=0)
    grouped_feature_variances = feature_variances.view(-1, group_size).mean(1)
    grouped_idx = grouped_feature_variances.argsort().flip(0)
    # will be # components / group_size
    idx = torch.cat([torch.arange(group_size).cuda() + (group_size*i) for i in grouped_idx])
    return feature_variances[idx], idx


In [ ]:
def plot_ordered_vs_random_acc_samples(ordered_accs, random_acc_sampling,
                                       baseline_ordered_accs=None, baseline_random_acc_sampling=None,
                                       title=None):
    random_accs = torch.mean(torch.stack(random_acc_sampling, dim=0), dim=0)
    plt.figure()
    plt.title(title)
    plt.ylabel('accuracy')
    plt.xlabel('# components used')
    plt.plot(ordered_accs.detach().cpu().numpy(), label="Sorted Variances")
    plt.plot(random_accs.detach().cpu().numpy(), label=f"Random (average {len(random_acc_sampling)} runs)")
    if baseline_ordered_accs is not None:
        plt.plot(baseline_ordered_accs.detach().cpu().numpy(), label="Baseline sorted")
    if baseline_random_acc_sampling is not None:
        plt.plot(torch.stack(baseline_random_acc_sampling).mean(0).detach().cpu().numpy(), label=f"Baseline Random (average {len(baseline_random_acc_sampling)} runs)")
    plt.legend()
    plt.show()
    
    
def generate_order_variance_plot(model, ds_name, preprocess, num_samples=2, group_size=1, normalization='mask',
                                    baseline_model=None):
    random_acc_sampling = [test_on_dataset(model, ds_name, preprocess,
                                           test_components_mode='random_all',
                                          component_grouping=group_size,
                                          normalization=normalization) for _ in range(num_samples)]
    ordered_accs = test_on_dataset(model, ds_name, preprocess,
                                   test_components_mode='test_all', component_grouping=group_size,
                                          normalization=normalization)
    
    baseline_random_acc_sampling = None
    baseline_ordered_accs = None
    if baseline_model is not None:
        baseline_random_acc_sampling = [test_on_dataset(baseline_model, ds_name, preprocess,
                                               test_components_mode='random_all',
                                              component_grouping=group_size,
                                              normalization=normalization) for _ in range(num_samples)]
        baseline_ordered_accs = test_on_dataset(baseline_model, ds_name, preprocess,
                                       test_components_mode='test_all', component_grouping=group_size,
                                              normalization=normalization)
    plot_ordered_vs_random_acc_samples(ordered_accs, random_acc_sampling, title=ds_name,
                                      baseline_ordered_accs=baseline_ordered_accs,
                                      baseline_random_acc_sampling=baseline_random_acc_sampling)
    
    
def simple_ordered_plot(model, ds_name, preprocess, group_size=1, normalization='mask',
                                    baseline_model=None,
                       parameter_x_axis=False):
    ordered_accs = test_on_dataset(model, ds_name, preprocess,
                                   test_components_mode='test_all', component_grouping=group_size,
                                          normalization=normalization)
    
    baseline_random_acc_sampling = None
    baseline_ordered_accs = None
    if baseline_model is not None:
        baseline_performance = test_on_dataset(baseline_model, ds_name, preprocess,
                                       test_components_mode='standard', component_grouping=group_size,
                                              normalization=normalization)
    plt.figure()
    plt.title(ds_name)
    plt.ylabel('accuracy')
    plt.plot(ordered_accs.detach().cpu().numpy(), label="Experts CLIP (Ours)")
    if baseline_performance is not None:
        plt.axhline(baseline_performance, label="Baseline CLIP (CC12M) Performance",
                   linestyle='dashed',
                   color='gray')
    plt.legend()
    if parameter_x_axis:
        plt.xlabel("Percentage of parameters used")
        num_baseline_params = sum([p.numel() for p in baseline_model.visual.parameters()])
        total_new_params = sum([p.numel() for p in model.visual.parameters()])
        locs, labels = plt.xticks()
        # locs are # of groups being used out of num_models
        num_models = model.visual[-2].groups
        labels = [f"{int(100*(loc/num_models)*(total_new_params/num_baseline_params))}%"
                 for loc in locs]
        plt.xticks(locs, labels)
    else:
        plt.xlabel('# groups used')
    plt.show()


In [ ]:
test_on_dataset(cc_model, 'dtd', preprocess)


In [ ]:
test_on_dataset(model, 'dtd', preprocess, test_components_mode='test_all',
               component_grouping=64, normalization='grouped')


In [ ]:
test_on_dataset(model, 'dtd', preprocess, test_components_mode='all_indiv',
               component_grouping=64, normalization='mask')

In [ ]:
test_on_dataset(model, 'dtd', preprocess, test_components_mode='ensemble',
               component_grouping=64, normalization='grouped')

In [ ]:
test_on_dataset(model, 'aircraft', preprocess, test_components_mode='test_all',
               component_grouping=64, normalization='grouped')

In [ ]:
test_on_dataset(model, 'cifar10', preprocess, test_components_mode='all_indiv',
               component_grouping=64, normalization='mask')

In [ ]:
test_on_dataset(model, 'cifar10', preprocess, test_components_mode='test_all',
               component_grouping=64, normalization='mask')

In [ ]:
test_on_dataset(model, 'aircraft', preprocess, test_components_mode='all_indiv',
               component_grouping=64, normalization='mask')

In [ ]:
test_on_dataset(model, 'eurosat', preprocess, test_components_mode='all_indiv',
               component_grouping=64, normalization='mask')

In [ ]:
test_on_dataset(model, 'cifar100', preprocess, test_components_mode='test_all',
               component_grouping=64, normalization='grouped')

In [ ]:
test_on_dataset(model, 'cub', preprocess, test_components_mode='standard',
               component_grouping=64, normalization='grouped')

In [ ]:
test_on_dataset(model, 'cub', preprocess, test_components_mode='test_all',
               component_grouping=64, normalization='grouped')

In [ ]:
test_on_dataset(model, 'dtd', preprocess, test_components_mode='random_all',
               component_grouping=64, normalization='grouped')

In [ ]:
test_on_dataset(model, 'cifar10', preprocess, test_components_mode='test_all',
               component_grouping=64, normalization='grouped')

In [ ]:
generate_order_variance_plot(model, 'dtd', preprocess, group_size=64, normalization='grouped', baseline_model=cc_model)

In [ ]:
simple_ordered_plot(model, 'dtd', preprocess, group_size=64, normalization='grouped', baseline_model=cc_model)

In [ ]:
simple_ordered_plot(model, 'dtd', preprocess, group_size=8, normalization='mask', baseline_model=cc_model,
                   parameter_x_axis=True)

In [ ]:
for ds in ds_names:
    if ds=='mnist': continue
    simple_ordered_plot(model, ds, preprocess, group_size=8, normalization='mask', baseline_model=cc_model,
                   parameter_x_axis=True)

In [ ]:
generate_order_variance_plot(model, 'mnist', grayscale_preprocess, group_size=8, global_norm=False, baseline_model=cc_model)

In [ ]:
generate_order_variance_plot(model, 'eurosat', preprocess, group_size=8, global_norm=False, baseline_model=cc_model)

In [ ]:
generate_order_variance_plot(model, 'cars', preprocess, group_size=8, global_norm=False, baseline_model=cc_model)

In [ ]:
generate_order_variance_plot(model, 'food101', preprocess, group_size=8, global_norm=False, baseline_model=cc_model)

In [ ]:
generate_order_variance_plot(model, 'cifar10', preprocess, group_size=8, global_norm=False, baseline_model=cc_model)

In [ ]:
generate_order_variance_plot(model, 'cifar100', preprocess, group_size=8, global_norm=False, baseline_model=cc_model)

In [ ]:
generate_order_variance_plot(model, 'cub', preprocess, group_size=8, global_norm=False, baseline_model=cc_model)

In [ ]:
generate_order_variance_plot(model, 'gtsrb', preprocess, group_size=8, global_norm=False, baseline_model=cc_model)

In [ ]:
generate_order_variance_plot(model, 'flowers', preprocess, group_size=64, global_norm=False, baseline_model=cc_model)

In [ ]:
generate_order_variance_plot(model, 'aircraft', preprocess, group_size=8, global_norm=False, baseline_model=cc_model)

In [ ]:
generate_order_variance_plot(cc_model, 'aircraft', preprocess)

In [ ]:
ds_names = ['dtd', 'mnist', 'eurosat', 'cars', 'food101', 'cifar10',
           'cifar100', 'cub', 'gtsrb', 'flowers', 'aircraft']

In [ ]:
for ds_name in ds_names:
    for net in [model, cc_model]:
        print(ds_name)
        print(test_on_dataset(net, ds_name,
                              grayscale_preprocess if ds_name=='mnist' else preprocess))

In [ ]:
for ds_name in ds_names:
    print(ds_name, len(dataset_constructor(ds_name, 'val', preprocess)))